Start work with setuping working table

In [2]:
USE WarehouseStageArea;
DECLARE @CurrentTableInWork AS INT = 1;

UPDATE Integration.PackageExecutions
SET Message = 'ssis development in progress'
WHERE  PackageExecutionId = @CurrentTableInWork

select *
from Integration.PackageExecutions
where Message = 'ssis development in progress'

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.017

PackageExecutionId,UserName,LastSuccessfulExtractionDatetime,CurrentExtractionDateTime,TableName,Message,ValidFrom,ValidTo
1,oranta\bezvershuk_do,2010-01-01 00:00:00.0000000,2010-01-01 00:00:00.0000000,Uran.Claims,ssis development in progress,2021-11-11 22:24:58.1723669,9999-12-31 23:59:59.9999999


### General
| General      | VariableName |
| ----------- | ----------- |
| Name      | GetLastSuccesfulLoadDate       |
| CodePage      | 1251       |
| ResultSet      | Single row      |

### Result Set View
| ResultName      | VariableName |
| ----------- | ----------- |
| LastSuccessfulExtractionDatetime      | Claims::LastSuccessfulExtraction       |
| CurrentExtractionDateTime   | Claims::CurrentExtraction        |

In [4]:
-- execute SQL TASK GetLastSuccesfulLoadDate
-- codepage 1251
-- ResultSet: SINGLEROW

DECLARE @tableName NVARCHAR(250) = N'Uran.Claims'
EXEC [Integration].[sp_GetExtractionPeriod] @tableName


Commands completed successfully.

Total execution time: 00:00:00.010

LastSuccessfulExtractionDatetime,CurrentExtractionDateTime
2010-01-01 00:00:00.0000000,2021-11-11 00:00:00.0000000


На продуктовом сервере есть вспомогательная таблица для генерации кода.

Для схемы `dbo` достаточно вставить имя оперируемой таблицы 

``` sql
INSERT INTO [tmp].[IntegrationServicesInfo] ([TableSchema], [TableName], [LogTableSchema])
VALUES ('dbo', 'Claims', 'log')
```

Скрипт генерации селекта:

```sql
DECLARE @Script as nvarchar(max) = ''

SELECT @Script = [Script1Part1] + '
' + Script1Part2  + TableSchema + '.' + [TableName] + Script1Part3 + [LogTableSchema] + '.' + [TableName] + [Script1Part4]
FROM [Oberon].[tmp].[IntegrationServicesInfo]
WHERE Id = 1

print @Script
```
```sql
SELECT 'src.' + COLUMN_NAME + ',
'
FROM INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_NAME = 'Claims' AND TABLE_SCHEMA = 'dbo'
ORDER BY ORDINAL_POSITION
```

```sql
SELECT * 
FROM (
SELECT 'MainData.' + COLUMN_NAME + ',
' AS ColumnName
FROM INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_NAME = 'Claims' AND TABLE_SCHEMA = 'dbo'
) AS Main
UNION ALL
SELECT 	'MainData.LogCreateDate,' AS ColumnName
UNION ALL
SELECT 	'MainData.LogActionDate,' AS ColumnName
UNION ALL
SELECT 	'MainData.SourceIdentity,' AS ColumnName
UNION ALL
SELECT 	'MainData.LoadDatetime' AS ColumnName
```

Сгенерированный скрипт для одной таблицы можно сохранить сразу в файл с расширением SQL
